In [ ]:
#google colab code

In [ ]:
!apt-get install espeak-ng -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 41 not upgraded.
Need to get 4,526 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-10ubuntu0.1 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libespeak-ng1 amd64 1.50+dfsg-10ubuntu0.1 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 espeak-ng amd64 1.50+dfsg-1

In [ ]:
!pip install phonemizer pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.4/213.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 26.1 MB/s eta 0:00:00


In [ ]:
#multiple files

#!/usr/bin/env python3
"""
Phonemize German Datasets
- Processes train, train_20h, validation, and test files.
- Converts text to phonemes using Espeak-NG.
- Saves output to a new directory.
"""

import pandas as pd
from phonemizer import phonemize
import gc
import os
from tqdm import tqdm

# ============================================================================
# CONFIGURATION
# ============================================================================
# Input directory (where your _cleaned.tsv files are)
INPUT_DIR = "/content/temp"

# Output directory (where _phoneme.tsv files will be saved)
OUTPUT_DIR = "/content/temps"

# List of files to process
# Note: Ensure these filenames match exactly what is in your folder
FILES_TO_PROCESS = [
    "train_130_cleaned.tsv",
    "train_20h_cleaned.tsv",
    "validation_5_cleaned.tsv",
    "test_5_cleaned.tsv"
]

BATCH_SIZE = 1000
# ============================================================================

def process_file(filename):
    input_path = os.path.join(INPUT_DIR, filename)

    # Create output filename (e.g., train_cleaned.tsv -> train_phoneme.tsv)
    base_name = filename.replace("_cleaned.tsv", "")
    output_filename = f"{base_name}_phoneme.tsv"
    output_path = os.path.join(OUTPUT_DIR, output_filename)

    print(f"\n{'='*60}")
    print(f"Processing: {filename}")
    print(f"Saving to:  {output_filename}")
    print(f"{'='*60}")

    if not os.path.exists(input_path):
        print(f"❌ ERROR: File not found: {input_path}")
        return

    # Load Data
    df = pd.read_csv(input_path, sep='\t')

    # Filter valid sentences
    df = df[df['sentence'].notna()].reset_index(drop=True)
    total_rows = len(df)
    print(f"Loaded {total_rows:,} rows.")

    # Check if output already exists (resume capability)
    if os.path.exists(output_path):
        print("⚠️ Output file already exists. Overwriting...")
        os.remove(output_path)

    # Batch Processing
    total_batches = (total_rows + BATCH_SIZE - 1) // BATCH_SIZE
    first_batch = True

    # Initialize progress bar
    with tqdm(total=total_rows, unit="samples", desc="Phonemizing") as pbar:
        for batch_idx in range(0, total_rows, BATCH_SIZE):
            batch_end = min(batch_idx + BATCH_SIZE, total_rows)
            batch_df = df.iloc[batch_idx:batch_end].copy()

            try:
                # Phonemize the batch
                # njobs=4 runs parallel processes for speed
                batch_phonemes = phonemize(
                    batch_df['sentence'].tolist(),
                    language='de',
                    backend='espeak',
                    strip=True,
                    preserve_punctuation=False,
                    with_stress=False,
                    njobs=4
                )

                batch_df['phonemes'] = batch_phonemes

                # Remove empty phonemes
                batch_df = batch_df[batch_df['phonemes'] != ""]

                # We need to keep columns like 'duration_s' and 'path', so we just append 'phonemes'
                # Reorder columns to put phonemes next to sentence for readability
                cols = list(batch_df.columns)
                if 'sentence' in cols:
                    idx = cols.index('sentence')
                    cols.insert(idx + 1, cols.pop(cols.index('phonemes')))
                    batch_df = batch_df[cols]

                # Append to output file
                batch_df.to_csv(
                    output_path,
                    sep='\t',
                    index=False,
                    mode='w' if first_batch else 'a',
                    header=first_batch
                )

                first_batch = False

                # Clear memory
                del batch_df, batch_phonemes
                gc.collect()

                pbar.update(batch_end - batch_idx)

            except Exception as e:
                print(f"\n❌ Error in batch {batch_idx}: {e}")

    # Verify result
    if os.path.exists(output_path):
        final_df = pd.read_csv(output_path, sep='\t')
        print(f"\n✓ Completed: {output_filename}")
        print(f"  Total samples saved: {len(final_df):,}")
    else:
        print("\n❌ Failed to save output file.")

def main():
    # 1. Create Output Directory
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    print(f"Output directory ready: {OUTPUT_DIR}")

    # 2. Process each file
    for filename in FILES_TO_PROCESS:
        process_file(filename)

    print(f"\n{'='*60}")
    print("🎉 ALL FILES PROCESSED SUCCESSFULLY")
    print(f"{'='*60}")

if __name__ == "__main__":
    main()

Output directory ready: /content/temps

Processing: train_130_cleaned.tsv
Saving to:  train_130_phoneme.tsv
Loaded 80,470 rows.


Phonemizing: 100%|██████████| 80470/80470 [00:51<00:00, 1563.15samples/s]



✓ Completed: train_130_phoneme.tsv
  Total samples saved: 80,470

Processing: train_20h_cleaned.tsv
Saving to:  train_20h_phoneme.tsv
Loaded 12,385 rows.


Phonemizing: 100%|██████████| 12385/12385 [00:05<00:00, 2454.68samples/s]



✓ Completed: train_20h_phoneme.tsv
  Total samples saved: 12,385

Processing: validation_5_cleaned.tsv
Saving to:  validation_5_phoneme.tsv
Loaded 3,065 rows.


Phonemizing: 100%|██████████| 3065/3065 [00:01<00:00, 2159.03samples/s]



✓ Completed: validation_5_phoneme.tsv
  Total samples saved: 3,065

Processing: test_5_cleaned.tsv
Saving to:  test_5_phoneme.tsv
Loaded 3,103 rows.


Phonemizing: 100%|██████████| 3103/3103 [00:01<00:00, 1595.24samples/s]



✓ Completed: test_5_phoneme.tsv
  Total samples saved: 3,103

🎉 ALL FILES PROCESSED SUCCESSFULLY
